## 『本次練習內容』
#### 使用Xception backbone做 Trnasfer Learning


## 『本次練習目的』
  #### 了解如何使用Transfer Learning
  #### 了解Transfer Learning的優點，可以觀察模型收斂速度

##### 可以自行嘗試多種架構

In [1]:
import numpy as np
import tensorflow as tf
from keras import backend as K

from keras.models import Model
from keras.models import Sequential

from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import GlobalAveragePooling2D
from keras.layers import Input

from keras.applications.xception import Xception
from keras.applications.resnet import ResNet50

from keras.datasets import cifar10
from sklearn.preprocessing import OneHotEncoder

from keras.callbacks import ReduceLROnPlateau, EarlyStopping

In [2]:
input_tensor = Input(shape=(32, 32, 3))

In [3]:
# Xception 架構
model_Xception = Xception(include_top=False, weights='imagenet',
                          input_tensor=input_tensor,
                          pooling=None, classes=10)
model_Xception.summary()

Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 15, 15, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 15, 15, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 15, 15, 32)   0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

In [4]:
# Resnet 50 架構
model_Resnet50 = ResNet50(include_top=False, weights='imagenet',
                          input_tensor=input_tensor,
                          pooling=None, classes=10)
model_Resnet50.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 16, 16, 64)   256         conv1_conv[0][0]                 
___________________________________________________________________________________________

## 準備 Cifar 10 資料

In [5]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

## Normalize Data
def normalize(X_train,X_test):
    mean = np.mean(X_train,axis=(0,1,2,3))
    std = np.std(X_train, axis=(0, 1, 2, 3))
    X_train = (X_train-mean)/(std+1e-7)
    X_test = (X_test-mean)/(std+1e-7)
    return X_train, X_test
    
    
## Normalize Training and Testset    
x_train, x_test = normalize(x_train, x_test) 

## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

## 決定model種類

In [6]:
model = model_Xception

## 添加層數

In [7]:
x = model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.1)(x)

predictions = Dense(10,activation='softmax')(x)
model = Model(inputs=model.input, outputs=predictions,name=model.name.capitalize())

print(f'Model name : {model.name.capitalize()}\nModel deepth :{len(model.layers)}')

Model name : Xception
Model deepth :136


## 鎖定特定幾層不要更新權重

In [8]:
lock_num = 100
for i, layer in enumerate(model.layers):
    layer.trainable = False if i < lock_num else True
print(f'Lock top {lock_num} layers.')  

Lock top 100 layers.


## Training

In [9]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
callbacks = [ReduceLROnPlateau(monitor="val_accuracy", factor=0.5, patience=3, verbose=1, min_lr=0.0001),
              EarlyStopping(monitor = 'val_loss', patience=5, verbose=1)]

history = model.fit(x_train, y_train, validation_split=0.2, batch_size=128, epochs=100, callbacks=callbacks)

Epoch 1/100
313/313 [==============================] - 278s 437ms/step - loss: 1.7520 - accuracy: 0.3654 - val_loss: 1.0465 - val_accuracy: 0.6476
Epoch 2/100
313/313 [==============================] - 133s 425ms/step - loss: 0.8270 - accuracy: 0.7185 - val_loss: 0.9296 - val_accuracy: 0.6857
Epoch 3/100
313/313 [==============================] - 92s 293ms/step - loss: 0.6175 - accuracy: 0.7839 - val_loss: 0.8558 - val_accuracy: 0.7127
Epoch 4/100
313/313 [==============================] - 92s 294ms/step - loss: 0.4568 - accuracy: 0.8440 - val_loss: 0.8913 - val_accuracy: 0.7215
Epoch 5/100
313/313 [==============================] - 96s 309ms/step - loss: 0.3187 - accuracy: 0.8931 - val_loss: 1.0596 - val_accuracy: 0.7107
Epoch 6/100
313/313 [==============================] - 99s 317ms/step - loss: 0.2303 - accuracy: 0.9224 - val_loss: 1.0516 - val_accuracy: 0.7111
Epoch 7/100
313/313 [==============================] - 92s 295ms/step - loss: 0.1647 - accuracy: 0.9442 - val_loss: 1.2173

In [10]:
# 不鎖定的話
model = model_Xception

x = model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.1)(x)

predictions = Dense(10,activation='softmax')(x)
model = Model(inputs=model.input, outputs=predictions,name=model.name.capitalize())
print(f'Model name : {model.name.capitalize()}\nModel deepth :{len(model.layers)}')

model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
callbacks = [ReduceLROnPlateau(monitor="val_accuracy", factor=0.5, patience=3, verbose=1, min_lr=0.0001),
              EarlyStopping(monitor = 'val_loss', patience=5, verbose=1)]

history = model.fit(x_train, y_train, validation_split=0.2, batch_size=128, epochs=100, callbacks=callbacks)

Model name : Xception
Model deepth :136
Epoch 1/100
313/313 [==============================] - 130s 319ms/step - loss: 0.2503 - accuracy: 0.9402 - val_loss: 1.3294 - val_accuracy: 0.7106
Epoch 2/100
313/313 [==============================] - 101s 324ms/step - loss: 0.0973 - accuracy: 0.9694 - val_loss: 1.3079 - val_accuracy: 0.7198
Epoch 3/100
313/313 [==============================] - 98s 315ms/step - loss: 0.0843 - accuracy: 0.9728 - val_loss: 1.4052 - val_accuracy: 0.7093
Epoch 4/100
313/313 [==============================] - 97s 311ms/step - loss: 0.0760 - accuracy: 0.9752 - val_loss: 1.4375 - val_accuracy: 0.7128
Epoch 5/100
313/313 [==============================] - 92s 294ms/step - loss: 0.0710 - accuracy: 0.9770 - val_loss: 1.4271 - val_accuracy: 0.7232
Epoch 6/100
313/313 [==============================] - 93s 299ms/step - loss: 0.0564 - accuracy: 0.9824 - val_loss: 1.5066 - val_accuracy: 0.7106
Epoch 7/100
313/313 [==============================] - 92s 295ms/step - loss: 0.05

In [11]:
# 換一種試試
model = model_Resnet50

x = model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.1)(x)

predictions = Dense(10,activation='softmax')(x)
model = Model(inputs=model.input, outputs=predictions,name=model.name.capitalize())
print(f'Model name : {model.name.capitalize()}\nModel deepth :{len(model.layers)}')

lock_num = 100
for i, layer in enumerate(model.layers):
    layer.trainable = False if i < lock_num else True
print(f'Lock top {lock_num} layers.')    

model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
callbacks = [ReduceLROnPlateau(monitor="val_accuracy", factor=0.5, patience=3, verbose=1, min_lr=0.0001),
              EarlyStopping(monitor = 'val_loss', patience=5, verbose=1)]

history = model.fit(x_train, y_train, validation_split=0.2, batch_size=128, epochs=100, callbacks=callbacks)

Model name : Resnet50
Model deepth :179
Lock top 100 layers.
Epoch 1/100
313/313 [==============================] - 171s 385ms/step - loss: 1.9008 - accuracy: 0.3466 - val_loss: 2.8015 - val_accuracy: 0.2538
Epoch 2/100
313/313 [==============================] - 138s 443ms/step - loss: 1.3841 - accuracy: 0.5208 - val_loss: 1.7508 - val_accuracy: 0.4095
Epoch 3/100
313/313 [==============================] - 127s 408ms/step - loss: 1.2452 - accuracy: 0.5641 - val_loss: 1.8121 - val_accuracy: 0.4382
Epoch 4/100
313/313 [==============================] - 128s 410ms/step - loss: 1.1182 - accuracy: 0.6070 - val_loss: 1.9271 - val_accuracy: 0.3703
Epoch 5/100
313/313 [==============================] - 128s 410ms/step - loss: 1.0106 - accuracy: 0.6485 - val_loss: 1.8407 - val_accuracy: 0.3920
Epoch 6/100
313/313 [==============================] - 127s 408ms/step - loss: 0.9229 - accuracy: 0.6708 - val_loss: 3.2018 - val_accuracy: 0.2752

Epoch 00006: ReduceLROnPlateau reducing learning rate to

In [12]:
model = model_Resnet50

x = model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.1)(x)

predictions = Dense(10,activation='softmax')(x)
model = Model(inputs=model.input, outputs=predictions,name=model.name.capitalize())
print(f'Model name : {model.name.capitalize()}\nModel deepth :{len(model.layers)}')

model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
callbacks = [ReduceLROnPlateau(monitor="val_accuracy", factor=0.5, patience=3, verbose=1, min_lr=0.0001),
              EarlyStopping(monitor = 'val_loss', patience=5, verbose=1)]

history = model.fit(x_train, y_train, validation_split=0.2, batch_size=128, epochs=100, callbacks=callbacks)

Model name : Resnet50
Model deepth :179
Epoch 1/100
313/313 [==============================] - 190s 490ms/step - loss: 1.2168 - accuracy: 0.6094 - val_loss: 4.6770 - val_accuracy: 0.3239
Epoch 2/100
313/313 [==============================] - 134s 429ms/step - loss: 0.6732 - accuracy: 0.7700 - val_loss: 2.1842 - val_accuracy: 0.4929
Epoch 3/100
313/313 [==============================] - 143s 458ms/step - loss: 0.5534 - accuracy: 0.8105 - val_loss: 2.8671 - val_accuracy: 0.3476
Epoch 4/100
313/313 [==============================] - 129s 412ms/step - loss: 0.4628 - accuracy: 0.8398 - val_loss: 3.9671 - val_accuracy: 0.3958
Epoch 5/100
313/313 [==============================] - 129s 414ms/step - loss: 0.3653 - accuracy: 0.8741 - val_loss: 5.3116 - val_accuracy: 0.2818

Epoch 00005: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 6/100
313/313 [==============================] - 128s 410ms/step - loss: 0.2150 - accuracy: 0.9305 - val_loss: 2.9532 - val_accuracy: 0.51